In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import lm_eval
from lm_eval.utils import setup_logging
from lm_eval.models.huggingface import HFLM

In [2]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval().to(device)
# prompt = "Explain the importance of reinforcement learning in AI."
# inputs = tokenizer(prompt, return_tensors="pt").to(device)
# output = model.generate(
#     **inputs,
#     max_length=100
# )

# # Decode the output tokens into text
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# print("output:")
# print(generated_text)


# Optional: configure logging

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
setup_logging("DEBUG")

hf_model = HFLM(pretrained=model, tokenizer=tokenizer)

results = lm_eval.simple_evaluate(
    model=hf_model,
    tasks=["hellaswag"],   # List of task names
    num_fewshot=0,                 # Zero-shot by default
    batch_size=8,
    limit=16                     # Use full datasets
)

2025-04-30:14:06:55,405 WARNING  [lm_eval.models.huggingface:100] `pretrained` model kwarg is not of type `str`. Many other model arguments may be ignored. Please do not launch via accelerate or use `parallelize=True` if passing an existing model this way.
2025-04-30:14:06:55,406 DEBUG    [lm_eval.models.huggingface:492] Using model type 'causal'
2025-04-30:14:06:55,427 WARNING  [lm_eval.models.huggingface:286] Passed an already-initialized model through `pretrained`, assuming single-process call to evaluate() or custom distributed integration
2025-04-30:14:06:55,429 INFO     [lm_eval.evaluator:169] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-04-30:14:06:55,429 INFO     [lm_eval.evaluator:222] Using pre-initialized model
2025-04-30:14:06:56,221 DEBUG    [lm_eval.tasks:523] File _evalita-mp_ner_wn.yaml in /home/jaeyongjang/.conda/envs/spinquant/lib/python3.10/site-packages/lm_eval/tasks/evalita_llm 

In [ ]:
print(results['results'])

{'hellaswag': {'alias': 'hellaswag', 'acc,none': 0.4375, 'acc_stderr,none': 0.128086884574495, 'acc_norm,none': 0.5, 'acc_norm_stderr,none': 0.12909944487358055}}
